# Synthetic Data Generation with Markov Model

In [13]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [16]:
data = pd.read_csv('../data_for_markov_model/processed_SmartTag_(nearby).csv')
data

,Unnamed: 0,Length Packet,Length Header,Length MS Data,Length Service Data,CH 37,CH 38,CH 39,AD Manufacturer Specific,AD Flags,...,PDU ADV_NONCONN_IND,PDU ADV_SCAN_IND,PDU Other,CT 07,CT 12,CT Other,ST 3,ST 5,ST Other,Label
0,0,63,37,0,160,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
1,1,63,37,0,160,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
2,2,63,37,0,160,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
3,3,63,37,0,160,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
4,4,63,37,0,160,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24033,24033,63,37,0,160,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
24034,24034,63,37,0,160,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
24035,24035,63,37,0,160,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
24036,24036,63,37,0,160,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,SmartTag (nearby)
